In [2]:
import matplotlib.pyplot as plt
from datetime import datetime, timedelta, time
import scipy.io as sio
from mat4py import loadmat
from scipy.io import savemat
from scipy.interpolate import interp1d
import numpy as np
import os
import pandas as pd
import scipy
from scipy import signal
import math
import csv
import time
from os import path
from typing import Tuple
# import hrvanalysis
from hrv.classical import frequency_domain, time_domain, non_linear
import pywt
import tsfel
import biosppy
import pyhrv.time_domain as td
import pyhrv.frequency_domain as fd
import pyhrv.tools as tools



import warnings
warnings.simplefilter(action='ignore')

from pylab import rcParams
rcParams['figure.figsize'] = 15, 10

ImportError: cannot import name 'MutableMapping' from 'collections' (c:\Users\huxua\AppData\Local\Programs\Python\Python311\Lib\collections\__init__.py)

In [3]:
def interp_condition(stats, window_len, max_missing):
    #stats is a row of the stats file over each window with the following columns : 
                #'window','Start_time','End_time','file_duration','N_IBI','Number of ectopic RR',
                #'% of ectopic RR','N_missing','%_missing','SNR_db','subject_id','Dass_before',
                #'Dass_after','Q_before','Q_after'
    # window_len : minimum window lenght (in seconds)
    # max_missing : maximum % of missing data in a window (in %)
    
    bool_duration = stats['file_duration'] >= window_len
    bool_missing_data = stats['%_missing'] <= max_missing
    
#     print(bool_duration[0])
#     print(bool_missing_data[0])
    
    rejection = (bool_missing_data[0] == False) or (bool_duration[0]==False)
#     print(rejection)
    
    return rejection

def get_open_file(info,general_path,day,segment,subject_id):
    
    path = "{0}/{1}/IBI_{1}_seg_{2}_subject_{3}.csv".format(general_path,day,segment,subject_id)
    df_file = pd.read_csv(path)
    
#     i = df_file[df_file['Datetime'] == info['Start_time']]
#     j= df_file['Datetime'] == info['End_time']
    
#     print(i,j)
    seg = df_file[(df_file['Datetime'] > info['Start_time'])  & (df_file['Datetime'] < info['End_time'])]
    seg = seg.reset_index()
    
    return seg
    



# @ @
# Interpolates a set of RR interval signals to make them evenly spaced and returns the interpolated RR interval signals
def standard_RR_interpolation(RRR,TTT):
    
#     subject_id = filename.rpartition['sujet_'][2]
    R = RRR
    T = TTT
    
    RR_interp = np.copy(R)
    lim= len(RR_interp)
       
    T_interp = np.copy(T)
    
    RR_interp = pd.DataFrame(RR_interp)
    #T_interp = pd.DataFrame(T_interp)
    
    N_vect = list()
    interp_indices = list()
    
    for i in range(2,lim-1):
        
        T = pd.to_datetime(T)
        T_start = T[i-1]
        T_end = T[i]
        
        trou = (T_end-T_start)/np.timedelta64(1,'s')
#         print(trou)
        
        #RR_mean = 1

        if trou > 1.3:
            
        #************* Mean and std ************  
            if i <11:
                RR_mean = np.nanmean(R[0:i])
            else:
                RR_mean = np.nanmean(R[(i-10):i])
            
            #N = round(trou/RR_mean)
            #print(i,trou, RR_mean)
            N = math.floor(trou/RR_mean)
            #print(type(N))
            
            N_vect = N_vect + [N]  # keep N number of Nan in the gap
            interp_indices = interp_indices + [i]   #gap indice
            
    acc = 0
    #print(len(RR_interp))
    RR_interp = RR_interp.to_numpy()
    
    for i in range(len(N_vect)):
        a = np.empty((N_vect[i],1))
        a[:] = np.nan
        
        RR_interp = np.insert(RR_interp, interp_indices[i]+1,a,axis=0)

        
    
    # Filtrage 
    for i in range(2,len(RR_interp)):
        if RR_interp[i] < 0.3 or RR_interp[i]>1.3:
            RR_interp[i] = np.nan
        
    
    RR_pchip = pd.DataFrame(RR_interp).interpolate(method='pchip')
           
#     RR_pchip.to_csv("{0}/{1}_stan_spline.csv".format(save_filepath, filename), index=False)
    
    return(RR_pchip)


def RR_original_stats(R,T):
        
    RR_interp = np.copy(R)
    lim= len(RR_interp)
       
    T_interp = np.copy(T)
    
    RR_interp = pd.DataFrame(RR_interp)
    #T_interp = pd.DataFrame(T_interp)
    
    N_vect = list()
    interp_indices = list()
    
    
    # Get file duration
    T = pd.to_datetime(T)
    
    T_0 = T.min()
    T_last=T.max()

#     print(T_0,T_last)
#     file_duration=(T_last-T_0)/np.timedelta64(1,'s')
    duration=(T_last-T_0)
    file_duration=duration.total_seconds()
    
#     file_duration = time.strftime("%H:%M:%S", time.gmtime(file_duration))
#     print(file_duration)
    
    # Get missing data index
    for i in range(2,lim-1):
        
        T = pd.to_datetime(T)
        T_start = T[i-1]
        T_end = T[i]
        
        trou = (T_end-T_start)/np.timedelta64(1,'s')
        

        if trou > 1.3:   
        #************* Mean and std ************  
            if i <11:
                RR_mean = np.nanmean(R[0:i])
            else:
                RR_mean = np.nanmean(R[(i-10):i])
            

            N = math.floor(trou/RR_mean)
#             print(trou,N)
            
            N_vect = N_vect + [N]  # keep N number of Nan in the gap
            interp_indices = interp_indices + [i]   #gap indice
            
    acc = 0

    
    # STATS
    ect_original =pd.DataFrame([len(R[R > 1.3])+ len(R[R <0.3])]) 
    
    df=pd.DataFrame({'Starting_time':[T_0]})
#     df['Starting_time']=T_0
    df['End_time']=T_last
    df['file_duration']=file_duration
    df['N_IBI'] = lim
    
    
    df['N_of_ectopic_RR']=ect_original
    df['%_of_ectopic_RR'] =100*ect_original[0]/len(R)
    
    df['N_missing'] = sum(N_vect)
    df['%_missing'] = 100*sum(N_vect)/(lim+sum(N_vect))

    df['subject_id']= subject_id
#     df['filename']= filename
    
    return(df)
            
           
# stats_columns = ['file_duration','Number of ectopic RR','% of ectopic RR','N_missing','%_missing','SNR_db','subject_id','filename']



#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%



            
           

        
# @@
# Returns a list containing the cumulative time (in milliseconds) between heartbeats
def calculate_real_time(rrs):
    acc = 0
    real_time = []

    for rr in rrs:
        real_time.append(acc)
        acc = acc + rr

    return real_time


def calc_true_hr(rrs):
    hrs = []
    for rr in rrs:
        hr = 60 * 1000 / rr
        hrs.append(hr)
    return hrs

def calc_true_time_domain(rrs, log):
#     rrs = np.array(rr)
#     print(rr)

    output = pd.DataFrame()
            
        # Time Domain Features    
#     tdom = time_domain(rrs)
    tdom=td.time_domain(nni=rrs) # 时域特征
    plt.close()
        
        # Non linear Domain
    nl = non_linear(rrs) # 非线性特征
        
        # TSFEL statistical
    vect = rrs
    abs_energy =  tsfel.feature_extraction.features.abs_energy(vect)
    autocorr =  tsfel.feature_extraction.features.autocorr(vect) 
    mean =  tsfel.feature_extraction.features.calc_mean(vect) 
    median =  tsfel.feature_extraction.features.calc_median(vect) 
    std =  tsfel.feature_extraction.features.calc_std(vect)
    entropy =  tsfel.feature_extraction.features.entropy(vect) 
    interq_range =  tsfel.feature_extraction.features.interq_range(vect) 
    kurtosis =  tsfel.feature_extraction.features.kurtosis(vect) 
    mean_abs_dev =  tsfel.feature_extraction.features.mean_abs_deviation(vect) 
    mean_abs_diff =  tsfel.feature_extraction.features.mean_abs_diff(vect) 
    mean_diff =  tsfel.feature_extraction.features.mean_diff(vect) 
    median_abs_dev =  tsfel.feature_extraction.features.median_abs_deviation(vect) 
    median_abs_diff =  tsfel.feature_extraction.features.median_abs_diff(vect) 
    median_diff =  tsfel.feature_extraction.features.median_diff(vect) 
    skewness =  tsfel.feature_extraction.features.skewness(vect) 
    slope =  tsfel.feature_extraction.features.slope(vect) 

    tsfel_feat = {'abs_energy':abs_energy,'autocorr':autocorr,'mean':mean,'median':median,'std':std,'entropy':entropy,
                  'interq_range':interq_range,'mean_abs_dev':mean_abs_dev,'mean_abs_diff':mean_abs_diff,
                  'mean_diff':mean_diff,'median_abs_dev':median_abs_dev,'median_abs_diff':median_abs_diff,
                  'median_diff':median_diff, 'kurtosis':kurtosis,'skewness':skewness,'slope':slope}

        # TSFEL frequency
        
    up_sampled_signal = up_sample(rrs, 8)
    vect = up_sampled_signal
        
    bw = tsfel.feature_extraction.features.power_bandwidth(vect, 8)
    spec_ent = tsfel.feature_extraction.features.spectral_entropy(vect, 8)
    spec_centroid = tsfel.feature_extraction.features.spectral_centroid(vect, 8)
    spec_decr = tsfel.feature_extraction.features.spectral_decrease(vect, 8)
    spec_dist = tsfel.feature_extraction.features.spectral_distance(vect, 8)
    spec_kurt = tsfel.feature_extraction.features.spectral_kurtosis(vect, 8)
    spec_roll_95 = tsfel.feature_extraction.features.spectral_roll_off(vect, 8)
    spec_skew = tsfel.feature_extraction.features.spectral_skewness(vect, 8)
    spec_var = tsfel.feature_extraction.features.spectral_variation(vect, 8)  
    energy =  tsfel.feature_extraction.features.total_energy(vect, 8)
    freq =  tsfel.feature_extraction.features.fundamental_frequency(vect, 8)
    med_freq =  tsfel.feature_extraction.features.median_frequency(vect, 8)
    auc =  tsfel.feature_extraction.features.auc(vect, 8) 
    h_e =  tsfel.feature_extraction.features.human_range_energy(vect, 8) 
        
        

    # 上采样后的信号中提取一系列频域特征
    tsfel_feat_spec = {'bw':bw,'spec_ent':spec_ent,'spec_decr':spec_decr,' spec_dist': spec_dist,'spec_kurt':spec_kurt,
                       ' spec_roll_95': spec_roll_95,'spec_skew':spec_skew,'spec_var':spec_var,'spec_centroid':spec_centroid,
                       'energy':energy,'fundamental_freq':freq,'med_freq':med_freq,'auc':auc,'human_energy':h_e}
    

#     time = []

#     for i in range(0, len(rrs)):
#         time.append(float(i))
#     print(time)

    fdom = frequency_domain(rrs,interp_method=None)

#         tps = {'Tps': j - 5}

    output = output.append({**tdom,**tsfel_feat, **nl, **fdom,**tsfel_feat_spec},ignore_index=True)
#         output = output.append({**tps, **tdom, **nl, **fdom}, ignore_index=True)

    return output


def up_sample(rr_intervals, frequency):
    time = calculate_real_time(rr_intervals)

    cs = interp1d(time, rr_intervals)
    xs = np.arange(0, time[-1], 1000 / frequency)

    cubic_spline = cs(xs)

    return cubic_spline


## This is the only funtion to call to compute HRV features
def hrv_features(RR, log=False):
    
    # RR should be in ms
    rr = RR.apply(lambda x: 1000 * x)
    foo=rr.to_numpy().flatten()
    
    df_HRV_features = calc_true_time_domain(foo, log)    

    return df_HRV_features
        

In [4]:

window_len= 300
max_missing=50 # Maximum percentage of acceptable missing data in each window to compute HRV features
wind = 300
step=60 #overlap of 1min = 60s

general_path = "C:/Users/huxua/Desktop/TX/Dataset2"
# general_path = "C:/Users/mouna/Desktop/experiment/Data/2_preprocessed_data/HRV/Signal_HRV_ECG"

save_path = "C:/Users/huxua/Desktop/TX/Dataset1"
# try:
#     os.mkdir(save_path)
# except OSError:
#     print('Creation failed or File already exists')

# save_file = "{0}/HRV_{1}_s_{2}_percent_step{3}.csv".format(save_path,window_len,max_missing,step)

global df_features

for subfolder in os.listdir('C:/Users/huxua/Desktop/TX/Dataset2'): # path pour les fichiers HRV
    for filename in os.listdir('C:/Users/huxua/Desktop/TX/Dataset2/'+subfolder):
        open_path = ('C:/Users/huxua/Desktop/TX/Dataset2/'+subfolder +'/'+filename)

#       #Remove extension xlsx from filename
        file = filename.rpartition('.csv')[0]
#             print(file)
        subject_id = filename.rpartition('_drowsy')[0]
                
#         filepath = "{0}/{1}".format(open_path,filename)
            
        df_file = pd.read_csv('{0}'.format(open_path))

        RR = df_file.RR_ST
        T = df_file.RR_timestamps_ST
        session = 0
        T = pd.to_datetime(T)

        begin, end = T.min(), T.max()
        print('begin, end',begin, end)
          
        shift = np.timedelta64(step,'s')   # same as datetime unit
        window = np.timedelta64(wind,'s')
        print('window',window)
        print('i=')
        for i in range(int((end - begin - window)/shift) + 1):
            print(i)

        ## Split Signal into 5min windows
        if len(RR)>10:

            buffers_limit = [[begin + i * shift, begin + window + i * shift] for i in range(int((end - begin - window)/shift) + 1,1)]
            buffers =  [df_file[(T >= buffer[0]) & (T < buffer[1])] for buffer in buffers_limit]

            j=0
            for buffer in buffers: 
                try:
                    RR = buffer.RR_ST.to_numpy()
                    T = buffer.RR_timestamps_ST.to_numpy()
                
                    df_stats = RR_original_stats(RR,T)
                    rejection = interp_condition(df_stats, window_len, max_missing)
                    
                    print('df_stats',df_stats)
                    print('rejection',rejection)
          
                    ## This is in order to reject segments with more than 50% of missing data
                    if rejection==False:
                
                        interp_RR = standard_RR_interpolation(RR,T)  #interpolation
                        df_features = hrv_features(interp_RR, log=False)     #compute HRV features
            
                        df_features['ID'] = subject_id
                        df_features['session'] = session

                        print('df inner',df_features)
                    
                    #Add features to csv database
                        save_file = ('{0}/HRV_ECG_step{1}_mona.csv'.format(save_path,step))
                        with open(save_file,'a') as f:
                            writer = csv.writer(f)
                            writer.writerow(df_features.iloc[0])
                            print('complete')
                except:
                    print('error')

'''save_file = ('{0}/HRV_ECG_step{1}_mona.csv'.format(save_path,step))
with open(save_file,'a') as f:
    writer = csv.writer(f)
    writer.writerow(df_features.columns)
    print('complete')'''

begin, end 1970-01-01 00:27:42.465388795 1970-01-01 00:27:42.471885321
window 300 seconds
i=
df_stats                   Starting_time                      End_time  file_duration  \
0 1970-01-01 00:27:42.465388795 1970-01-01 00:27:42.471885321       0.006496   

   N_IBI  N_of_ectopic_RR  %_of_ectopic_RR  N_missing  %_missing subject_id  
0   6314             6314            100.0          0        0.0        333  
rejection True
df_stats                   Starting_time                      End_time  file_duration  \
0 1970-01-01 00:27:42.465388795 1970-01-01 00:27:42.471885321       0.006496   

   N_IBI  N_of_ectopic_RR  %_of_ectopic_RR  N_missing  %_missing subject_id  
0   6314             6314            100.0          0        0.0        333  
rejection True
df_stats                   Starting_time                      End_time  file_duration  \
0 1970-01-01 00:27:42.465388795 1970-01-01 00:27:42.471885321       0.006496   

   N_IBI  N_of_ectopic_RR  %_of_ectopic_RR  N_missing  

"save_file = ('{0}/HRV_ECG_step{1}_mona.csv'.format(save_path,step))\nwith open(save_file,'a') as f:\n    writer = csv.writer(f)\n    writer.writerow(df_features.columns)\n    print('complete')"